# Time to Trial End
Determine the value of hyperparameter `cnfg.TIME_TO_TRIAL_END_THRESHOLD`, which is used to decide if a fixation / visit is a _LWS_ instance or not.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import config as cnfg

# pio.renderers.default = "notebook"
pio.renderers.default = "browser"

### Read data

In [2]:
from analysis.pipeline.full_pipeline import read_saved_data

_targets, actions, _metadata, idents, fixations, _visits = read_saved_data()

### Action-Time to Trial End

In [3]:
percentiles = [0.01, 0.05, 0.1, 0.25, 0.5]

action_summary = (
    pd.concat([
        actions["to_trial_end"].describe(percentiles).rename("all"),
        actions.groupby("subject")["to_trial_end"].describe(percentiles).T,
    ], axis=1)
).T

action_summary

,count,mean,std,min,1%,5%,10%,25%,50%,max
all,1515.0,11044.120132,5846.135507,253.0,756.00,1507.70,2959.2,6269.00,10971.0,27739.0
2,121.0,12218.082645,4918.294797,1363.0,1445.00,3410.00,5058.0,8925.00,12483.0,21783.0
3,111.0,12119.882883,5411.021103,253.0,511.80,1931.00,5077.0,7898.50,12765.0,20721.0
12,120.0,12951.483333,5338.934483,901.0,1212.19,1968.15,4861.4,9245.25,14228.5,21703.0
13,106.0,7765.358491,3475.230964,545.0,1350.70,2841.75,3600.0,4974.00,7855.5,15523.0
14,110.0,12792.163636,5649.506374,692.0,828.39,2409.55,4170.3,8606.25,13607.5,24268.0
15,107.0,6914.710280,5574.404021,355.0,861.44,1027.30,1092.0,1756.00,4948.0,19219.0
16,93.0,12862.182796,6126.009344,910.0,1419.68,2257.60,3084.2,7237.00,14411.0,22835.0
17,135.0,8390.488889,5372.670041,445.0,529.60,986.50,2109.0,3978.50,7708.0,26553.0
18,137.0,11592.145985,5566.565899,320.0,564.56,2308.20,4135.0,7267.00,11565.0,21730.0


In [4]:
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, shared_yaxes=False,
)

# top: distribution across all subjects
fig.add_trace(
    row=1, col=1, trace=go.Violin(
        y0="to_trial_end", x=actions["to_trial_end"],
        name="All Subjects", legendgroup="All Subjects",
        text=actions.apply(
            lambda row: f"Subject: {row['subject']}<br>"
                        f"Trial: {row['trial']}<br>"
                        # f"Target: {row['target']}<br>"
                        f"Time to Trial End: {row['to_trial_end']:.2f} DVA",
            axis=1
        ),
        marker=dict(color=cnfg.get_discrete_color("all")),
        width=1.75, orientation="h", side="positive", spanmode='hard',
        box=dict(visible=False),
        meanline=dict(visible=True),
        points="all", pointpos=-0.5,
        showlegend=True, hoverinfo="x+y+text",

    )
)

# bottom: distribution per subject
for subj_id in actions[cnfg.SUBJECT_STR].unique():
    subj_string = f"{cnfg.SUBJECT_STR.capitalize()} {subj_id:02d}"
    subj_data = actions[actions[cnfg.SUBJECT_STR] == subj_id]
    texts = subj_data.apply(
        lambda row: f"{subj_string}<br>"
                    f"Trial: {row['trial']}<br>"
                    # f"Target: {row['target']}<br>"
                    f"Time to Trial End: {row['to_trial_end']:.2f} DVA",
        axis=1
    )
    fig.add_trace(
        row=2, col=1, trace=go.Violin(
            y0="to_trial_end", x=subj_data["to_trial_end"],
            text=texts,
            name=subj_string, legendgroup=subj_string,
            marker=dict(color=cnfg.get_discrete_color(subj_id, loop=True), opacity=0.5),
            width=1.75, orientation="h", side="positive", spanmode='hard',
            box=dict(visible=False),
            meanline=dict(visible=True),
            points="all", pointpos=-0.5,
            showlegend=True, hoverinfo="x+y+text"
        )
    )

# update visuals
fig.update_annotations(font=cnfg.AXIS_LABEL_FONT)
fig.update_yaxes(showticklabels=False)  # Hide y-axis labels
fig.update_xaxes(
    title=None, showline=False,
    showgrid=True, gridcolor=cnfg.GRID_LINE_COLOR, gridwidth=cnfg.GRID_LINE_WIDTH,
    zeroline=False, zerolinecolor=cnfg.GRID_LINE_COLOR, zerolinewidth=cnfg.ZERO_LINE_WIDTH,
    tickfont=cnfg.AXIS_TICK_FONT,
)
fig.update_layout(
    width=1200, height=675,
    title=dict(text="Identification-Action to Trial End", font=cnfg.TITLE_FONT),
    paper_bgcolor='rgba(0, 0, 0, 0)',
    # plot_bgcolor='rgba(0, 0, 0, 0)',
    showlegend=True,
)

fig.show()

### Identification-Fixation to Trial End

In [5]:
fixs_with_ident_time = fixations.copy()
dva_cols = [col for col in fixations.columns if col.endswith("distance_dva")]
fixs_with_ident_time["target"] = fixs_with_ident_time[dva_cols].idxmin(axis=1).str.replace("_distance_dva", "")
fixs_with_ident_time = (
    fixs_with_ident_time
    .drop(columns=[col for col in fixs_with_ident_time.columns if "_distance_" in col])
    .merge(
        idents.loc[
            idents["identification_category"] == "hit", ["subject", "trial", "target", "time"]
        ], on=["subject", "trial", "target"], how="left"
    )
)

fixs_with_ident_time.loc[:, "is_during"] = (fixs_with_ident_time["start_time"] <= fixs_with_ident_time["time"]) & (fixs_with_ident_time["time"] <= fixs_with_ident_time["end_time"])

In [6]:
ident_fixs = fixs_with_ident_time.loc[fixs_with_ident_time["is_during"]]
ident_fixs_dist_summary = (
    pd.concat([
        ident_fixs["to_trial_end"].describe(percentiles).rename("all"),
        ident_fixs.groupby("subject")["to_trial_end"].describe(percentiles).T,
    ], axis=1)
).T

print("Identification Fixations:")
ident_fixs_dist_summary

Identification Fixations:


,count,mean,std,min,1%,5%,10%,25%,50%,max
all,2227.0,10609.801527,5698.764967,5.0,285.00,1116.40,2596.4,6178.50,10458.0,26464.0
2,160.0,12023.043750,4909.490862,784.0,859.21,3283.80,5378.6,8480.00,12052.0,21298.0
3,198.0,11803.424242,5163.462849,5.0,706.31,2599.50,5291.4,7646.00,12378.0,20212.0
12,136.0,12446.639706,4867.806654,651.0,878.80,3399.75,6062.5,9497.00,13019.5,19258.0
13,172.0,7252.860465,3213.377348,496.0,1058.32,2816.05,3126.6,4559.25,7106.0,14982.0
14,183.0,12908.524590,5083.491259,252.0,525.88,3410.20,6001.6,9455.00,13592.0,24121.0
15,160.0,6170.781250,5616.742473,5.0,222.12,433.00,538.9,901.75,4382.0,18750.0
16,180.0,12415.605556,6002.539472,51.0,769.32,1829.00,2945.6,6967.50,14323.0,22212.0
17,166.0,7608.596386,5220.803326,35.0,162.75,446.75,1359.0,3513.75,6674.0,26464.0
18,233.0,11214.909871,5537.665372,6.0,269.64,1528.00,3848.4,6928.00,11356.0,20583.0


In [7]:
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, shared_yaxes=False,
)

# top: distribution across all subjects
fig.add_trace(
    row=1, col=1, trace=go.Violin(
        y0="to_trial_end", x=ident_fixs["to_trial_end"],
        name="All Subjects", legendgroup="All Subjects",
        text=ident_fixs.apply(
            lambda row: f"Subject: {row['subject']}<br>"
                        f"Trial: {row['trial']}<br>"
                        # f"Target: {row['target']}<br>"
                        f"Time to Trial End: {row['to_trial_end']:.2f} DVA",
            axis=1
        ),
        marker=dict(color=cnfg.get_discrete_color("all")),
        width=1.75, orientation="h", side="positive", spanmode='hard',
        box=dict(visible=False),
        meanline=dict(visible=True),
        points="all", pointpos=-0.5,
        showlegend=True, hoverinfo="x+y+text",

    )
)

# bottom: distribution per subject
for subj_id in ident_fixs[cnfg.SUBJECT_STR].unique():
    subj_string = f"{cnfg.SUBJECT_STR.capitalize()} {subj_id:02d}"
    subj_data = ident_fixs[ident_fixs[cnfg.SUBJECT_STR] == subj_id]
    texts = subj_data.apply(
        lambda row: f"{subj_string}<br>"
                    f"Trial: {row['trial']}<br>"
                    # f"Target: {row['target']}<br>"
                    f"Time to Trial End: {row['to_trial_end']:.2f} DVA",
        axis=1
    )
    fig.add_trace(
        row=2, col=1, trace=go.Violin(
            y0="to_trial_end", x=subj_data["to_trial_end"],
            text=texts,
            name=subj_string, legendgroup=subj_string,
            marker=dict(color=cnfg.get_discrete_color(subj_id, loop=True), opacity=0.5),
            width=1.75, orientation="h", side="positive", spanmode='hard',
            box=dict(visible=False),
            meanline=dict(visible=True),
            points="all", pointpos=-0.5,
            showlegend=True, hoverinfo="x+y+text"
        )
    )

# update visuals
fig.update_annotations(font=cnfg.AXIS_LABEL_FONT)
fig.update_yaxes(showticklabels=False)  # Hide y-axis labels
fig.update_xaxes(
    title=None, showline=False,
    showgrid=True, gridcolor=cnfg.GRID_LINE_COLOR, gridwidth=cnfg.GRID_LINE_WIDTH,
    zeroline=False, zerolinecolor=cnfg.GRID_LINE_COLOR, zerolinewidth=cnfg.ZERO_LINE_WIDTH,
    tickfont=cnfg.AXIS_TICK_FONT,
)
fig.update_layout(
    width=1200, height=675,
    title=dict(text="Identification-Fixation to Trial End", font=cnfg.TITLE_FONT),
    paper_bgcolor='rgba(0, 0, 0, 0)',
    # plot_bgcolor='rgba(0, 0, 0, 0)',
    showlegend=True,
)

fig.show()

### Visit to Trial End

In [8]:
pd.concat([
    _visits["to_trial_end"].describe(percentiles).rename("all"),
    _visits.groupby("subject")["to_trial_end"].describe(percentiles).T,
], axis=1).T

,count,mean,std,min,1%,5%,10%,25%,50%,max
all,6072.0,10183.841733,5857.623433,5.0,32.39,944.85,2085.5,5357.00,9945.0,26466.0
2,696.0,10143.827586,5087.732663,5.0,508.95,1284.00,2880.5,6681.25,10124.0,21298.0
3,486.0,10251.201646,6009.157113,5.0,5.85,1089.50,1880.0,5455.50,9878.0,22016.0
12,453.0,12364.830022,5630.274390,158.0,650.04,1833.60,3100.4,8521.00,13356.0,21303.0
13,443.0,7054.884876,3723.349991,5.0,146.26,1196.00,2812.2,4283.50,6579.0,16209.0
14,395.0,10639.794937,6050.980651,5.0,6.00,654.60,1845.0,5795.50,10819.0,24805.0
15,368.0,7942.953804,5931.198148,5.0,374.34,532.85,677.0,2625.75,6910.0,20070.0
16,463.0,11676.954644,6306.685892,5.0,5.00,983.60,2003.0,6286.50,13169.0,22212.0
17,353.0,8141.818697,5431.031194,35.0,250.24,774.60,1611.0,3701.00,7264.0,26106.0
18,575.0,10728.996522,5953.207482,5.0,50.82,1062.50,2273.8,6182.00,10114.0,22631.0
